In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

ModuleNotFoundError: No module named 'findspark'

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()

In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
fourbed_query = 
"""SELECT YEAR(date), ROUND(AVG(price),2)
    FROM home_sales WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY YEAR(date) DESC"""

spark.sql(fourbed_query).show()

In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
Threebedbath = 
"""SELECT date_built, ROUND(AVG(price),2)
    FROM home_sales WHERE bedrooms = 3 and bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built desc"""

spark.sql(Threebedbath).show()


In [ ]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
Threebedbathtwostory = 
"""SELECT date_built, ROUND(AVG(price),2)
    FROM home_sales WHERE bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >=2000
    GROUP BY date_built
    ORDER BY date_built DESC"""

spark.sql(Threebedbathtwostory).show()

In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
rating = 
"""SELECT view, ROUND(AVG(price),2)
    FROM home_sales WHERE price >=350000
    GROUP BY view
    ORDER BY view DESC"""


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

cache = 
"""SELECT view ROUND(AVG(price),2)
    FROM home_sales
    GROUP BY view
    ORDER BY view"""

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy('date_built').parquet('home_sales_parq',mode='overwrite')

In [ ]:
# 11. Read the formatted parquet data.
parquet_homes_df = spark.read.parquet("home_sales_parq")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_homes_df = createOrReplaceTempView('home_sales_parq')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

parq_query = 
"""SELECT view, ROUND(AVG(price),2)
    FROM home_sales_parq WHERE price >=350000
    GROUP BY view
    ORDER BY view DESC"""

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
    print('home_sales cached')
else:
    print('no cached tables')
